In [ ]:
##############################################################################
## Fundamentals for pratical Text Analytics - spacy for NLP tasks
##                                            NER, 
##                                            use models to extract objects in our text
##                                            why:  reporting/summarization
##                                                  use downstream (API calls, db lookups)
##
##
## Learning goals:
##                 - SPACY!
##                 - NER extraction
##                 - annotating our own data is a real thing
##
##
## Great resources
##                 - https://spacy.io/usage/spacy-101
##
##############################################################################

In [ ]:
# ! pip install -U textacy

In [ ]:
# installs
# ! pip install -U spacy 
# ! pip install -U textacy
# ! pip install newspaper3k
# ! pip install afinn

In [5]:
# imports
import spacy
from spacy import cli
from spacy import displacy
import pandas as pd


# upcoming!
from textacy.extract.keyterms import textrank
import gensim

import textacy

from newspaper import Article
import json

from afinn import Afinn


# Spacy

In [ ]:
######################################### lets get started with spacy!
##
## "industrial strength" NLP tools
## increasingly the go to in this space, but like Textblob, 
## it focuses on document-oriented operations
## lots of configuration and power, but lets get the foundations set
##

![](https://d33wubrfki0l68.cloudfront.net/3ad0582d97663a1272ffc4ccf09f1c5b335b17e9/7f49c/pipeline-fde48da9b43661abcdf62ab70a546d71.svg)

In [19]:
# If VS Code crashes
# use
# (from command line python -m spacy download en_core_web_md)
# and ignore the cli commands below, proceed to spacy.load

model = "en_core_web_md"
cli.download(model)

nlp = spacy.load('en_core_web_md')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# spacy operates via language models
# https://spacy.io/models
# we setup the model as nlp, the spacy convention


In [3]:
# the message we will parse into a document
msg = 'BA820 uses python and spacy.'

# we are using all of the defaults, but this is a "heavy" parse with a ton of great data to explore
# https://spacy.io/usage/spacy-101#features

doc = nlp(msg)

In [4]:
# what do we have?
type(doc)

spacy.tokens.doc.Doc

In [6]:
# we can list it, but those are not ordinary pieces of text!
list(doc)

[BA820, uses, python, and, spacy, .]

In [7]:
# tokens
token = doc[0]
token

BA820

In [ ]:
# the tokens have properties/attributes
# https://spacy.io/usage/linguistic-features#tokenization

![](https://spacy.io/tokenization-9b27c0f6fe98dcb26239eba4d3ba1f3d.svg)

In [8]:
# link to pseudocode for how spacy tokenizes 
# https://spacy.io/usage/linguistic-features#how-tokenizer-works
[token.text for token in doc]

['BA820', 'uses', 'python', 'and', 'spacy', '.']

In [ ]:
# what do we have with a token?
# lots of nlp elements!


In [9]:
# attributes
# https://spacy.io/api/token
token.is_lower
token.is_punct
token.is_stop


False

In [10]:
# go back to the doc
len(doc)

6

In [ ]:
# extacting the tokenization in spacy


In [13]:
# we saw above, each token is an object we can inspect
# remove punct
[token.text for token in doc if not token.is_punct]

# make lower case
[token.text.lower() for token in doc if not token.is_punct]


['ba820', 'uses', 'python', 'and', 'spacy']

In [12]:
# remove stopwords
# good examples on accessing/modifying stopwords:
# https://stackoverflow.com/questions/41170726/add-remove-custom-stop-words-with-spacy

[token.text.lower() for token in doc if not token.is_punct and not token.is_stop]

['ba820', 'uses', 'python', 'spacy']

In [ ]:
# lets just keep everything as is


In [15]:
parsed = [(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.head) for token in doc]
parsed


[('BA820', 'BA820', 'PROPN', 'NNP', 'nsubj', uses),
 ('uses', 'use', 'VERB', 'VBZ', 'ROOT', uses),
 ('python', 'python', 'NOUN', 'NN', 'dobj', uses),
 ('and', 'and', 'CCONJ', 'CC', 'cc', python),
 ('spacy', 'spacy', 'NOUN', 'NN', 'conj', python),
 ('.', '.', 'PUNCT', '.', 'punct', uses)]

In [17]:
# ok, there is a lot going on, but we can now see the idea of the language models in action
# we are tokenizing text
# the text itself has representations, like pos, lemma, etc.
# https://spacy.io/usage/linguistic-features#pos-tagging

# luckily, spacy has built in help!
spacy.explain('PROPN')
spacy.explain('CCONJ')


'coordinating conjunction'

In [18]:
# spacy, via it's models (learned from labeled data!) can even map the relationships


# for those in jupyter or colab
# displacy.render(doc, style="dep", jupyter=True)

# if in vs code - spacy has a built in server
# the viz is at http://localhost:5000 
displacy.serve(doc, style="dep")

/Users/alexandervonschwerdtner/opt/miniconda3/envs/ba820/lib/python3.8/site-packages/spacy/displacy/__init__.py:98: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [29/Nov/2021 10:18:18] "GET / HTTP/1.1" 200 4236
127.0.0.1 - - [29/Nov/2021 10:18:19] "GET /favicon.ico HTTP/1.1" 200 4236


Shutting down server on port 5000.


In [19]:
#######################################  YOUR TURN
# using the corpus below, parse the text and POS tag, and lemma

corpus = ["Brock likes golf and hockey",
          "I love teaching at Questrom",
          "Python makes data analytics fun!"]

In [37]:
parsed = []

for d in corpus:
    doc = nlp(d)
    for token in doc:
        parsed.append((token.text, token.lemma_, token.pos_))
        
        
parsed

[('Brock', 'Brock', 'PROPN'),
 ('likes', 'like', 'VERB'),
 ('golf', 'golf', 'NOUN'),
 ('and', 'and', 'CCONJ'),
 ('hockey', 'hockey', 'NOUN'),
 ('I', 'I', 'PRON'),
 ('love', 'love', 'VERB'),
 ('teaching', 'teaching', 'NOUN'),
 ('at', 'at', 'ADP'),
 ('Questrom', 'Questrom', 'PROPN'),
 ('Python', 'Python', 'PROPN'),
 ('makes', 'make', 'VERB'),
 ('data', 'data', 'NOUN'),
 ('analytics', 'analytic', 'NOUN'),
 ('fun', 'fun', 'NOUN'),
 ('!', '!', 'PUNCT')]

In [32]:
docs = list(nlp.pipe(corpus))
docs

[Brock likes golf and hockey,
 I love teaching at Questrom,
 Python makes data analytics fun!]

In [33]:
[(token.text, token.lemma_, token.pos_) for doc in docs for token in doc]

[('Brock', 'Brock', 'PROPN'),
 ('likes', 'like', 'VERB'),
 ('golf', 'golf', 'NOUN'),
 ('and', 'and', 'CCONJ'),
 ('hockey', 'hockey', 'NOUN'),
 ('I', 'I', 'PRON'),
 ('love', 'love', 'VERB'),
 ('teaching', 'teaching', 'NOUN'),
 ('at', 'at', 'ADP'),
 ('Questrom', 'Questrom', 'PROPN'),
 ('Python', 'Python', 'PROPN'),
 ('makes', 'make', 'VERB'),
 ('data', 'data', 'NOUN'),
 ('analytics', 'analytic', 'NOUN'),
 ('fun', 'fun', 'NOUN'),
 ('!', '!', 'PUNCT')]

In [25]:
corpus_str = str(corpus)

In [27]:
doc2 = nlp(corpus_str)
doc2

['Brock likes golf and hockey', 'I love teaching at Questrom', 'Python makes data analytics fun!']

In [30]:
parsed = [(token.text, token.lemma_, token.pos_) for token in doc2]
parsed

[('[', '[', 'X'),
 ("'", "'", 'PUNCT'),
 ('Brock', 'Brock', 'PROPN'),
 ('likes', 'like', 'VERB'),
 ('golf', 'golf', 'NOUN'),
 ('and', 'and', 'CCONJ'),
 ('hockey', 'hockey', 'NOUN'),
 ("'", "'", 'PUNCT'),
 (',', ',', 'PUNCT'),
 ("'", "'", 'PUNCT'),
 ('I', 'I', 'PRON'),
 ('love', 'love', 'VERB'),
 ('teaching', 'teaching', 'NOUN'),
 ('at', 'at', 'ADP'),
 ('Questrom', 'Questrom', 'PROPN'),
 ("'", "'", 'PART'),
 (',', ',', 'PUNCT'),
 ("'", "'", 'PUNCT'),
 ('Python', 'Python', 'PROPN'),
 ('makes', 'make', 'VERB'),
 ('data', 'data', 'NOUN'),
 ('analytics', 'analytic', 'NOUN'),
 ('fun', 'fun', 'NOUN'),
 ('!', '!', 'PUNCT'),
 ("'", "'", 'PUNCT'),
 (']', ']', 'PUNCT')]

![](https://hashouttech.com/static/3c5973a520b86c3660a9771453df5794/2bef9/span-object.png)

In [38]:
# before we dive into NER, its worth noting that docs are a brilliant representation for text
# at the core, and to keep simple, think like slicing elements of a list

doc

Python makes data analytics fun!

In [39]:
doc[:2]

Python makes

In [40]:
myspan = doc[:4]
myspan

Python makes data analytics

In [41]:
type(myspan)

spacy.tokens.span.Span

In [ ]:
# these are just, well, spans of a document
# think slices


In [ ]:
# many of the same elements apply


In [ ]:
# why is this important?
# a token, or span of tokens, could have meaning
# Named Entity Recognition - NER



# Named Entity Recognition (NER)

In [ ]:
#######################################  Named Entity Recognition
## 
## We have seen regex can be very powerful
## not only can we tokenize data, but we COULD use it to parse patterns
##
## HOWEVER:  the spacy parsing has already trained a GENERALIZED model for us
##           lets start there! But note, based on certain tasks, spacy is near/at SOTA
## 
## https://spacy.io/usage/linguistic-features#named-entities
## 
## Why does this matter?
## - we have large corpora and want to extract the entities being discussed
## - think legal documents -  which people/organizations are involved
## - news organizations tagging/categorizing articles to compare across all articles
## - content recommendations - other texts including this entity/entities
## - customer support - which products/services are our customers reference in service requests
## - medical - illnesses or diseases per medical intake forms
## - hiring/scanning: skill detection, experience detection
## - product names/ids

Spacy trained entities.

```
PERSON:      People, including fictional.
NORP:        Nationalities or religious or political groups.
FAC:         Buildings, airports, highways, bridges, etc.
ORG:         Companies, agencies, institutions, etc.
GPE:         Countries, cities, states.
LOC:         Non-GPE locations, mountain ranges, bodies of water.
PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
EVENT:       Named hurricanes, battles, wars, sports events, etc.
WORK_OF_ART: Titles of books, songs, etc.
LAW:         Named documents made into laws.
LANGUAGE:    Any named language.
DATE:        Absolute or relative dates or periods.
TIME:        Times smaller than a day.
PERCENT:     Percentage, including ”%“.
MONEY:       Monetary values, including unit.
QUANTITY:    Measurements, as of weight or distance.
ORDINAL:     “first”, “second”, etc.
CARDINAL:    Numerals that do not fall under another type.
```

> https://towardsdatascience.com/explorations-in-named-entity-recognition-and-was-eleanor-roosevelt-right-671271117218

In [42]:
msg = 'Questrom is a business school at Boston University in Massachusetts'

doc = nlp(msg)

doc

Questrom is a business school at Boston University in Massachusetts

In [43]:
# 'basic' python still applies
doc.ents

(Boston University, Massachusetts)

In [45]:
# we can grab these out
len(doc.ents)
ent = doc.ents
ent0 = ent[0]

ent0

Boston University

In [46]:
# these have properties/attributes
type(ent0)

spacy.tokens.span.Span

In [47]:
# remember explain?
ent0.start_char

33

In [48]:
# there is a visualizer too
# of course, we can visualize this.  spacy is the bees knees

# displacy.render(doc, style="ent", jupyter=True)

# or in vs code -- localhost:5000
displacy.serve(doc, style="ent")

/Users/alexandervonschwerdtner/opt/miniconda3/envs/ba820/lib/python3.8/site-packages/spacy/displacy/__init__.py:98: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [29/Nov/2021 11:09:11] "GET / HTTP/1.1" 200 1095


Shutting down server on port 5000.


In [57]:
# NER isn't perfect as we just saw
# another example

corpus = ["Let's go to the store", "I enjoy programming in the language go"]

goents = []
for d in corpus:
  doc = nlp(d)
  for ent in doc.ents:
    goents.append((ent.text, ent.label_)) 

goents


[]

/Users/alexandervonschwerdtner/opt/miniconda3/envs/ba820/lib/python3.8/site-packages/spacy/displacy/__init__.py:98: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/Users/alexandervonschwerdtner/opt/miniconda3/envs/ba820/lib/python3.8/site-packages/spacy/displacy/__init__.py:192: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [85]:
#######################################  YOUR TURN
##
## parse the article at the URL below
## trick: consider this a document, not a corpus
## extract the entities like we have above into a list
## visualize, if you can

URL = "https://www.lyrics.com/lyric/180684/Billy+Joel/We+Didn%27t+Start+the+Fire"

# HINT:  we have seen how easy the newspaper3k package makes extracting data from the web!
article = Article(URL)
article.download()
article.parse()

In [87]:
article = article.text

In [88]:
doc = nlp(article)

In [89]:
ents_song = []

for e in doc.ents:
    ents_song.append((e.text, e.start_char,e.end_char,e.label))
    
ents_song

[('Harry Truman', 0, 12, 380),
 ('Red China', 25, 34, 383),
 ('Johnnie Ray South Pacific', 36, 61, 380),
 ('Walter Winchell', 63, 78, 380),
 ('Joe DiMaggio Joe McCarthy', 80, 105, 380),
 ('Richard Nixon', 107, 120, 380),
 ('Studebaker', 122, 132, 383),
 ('North Korea', 145, 156, 384),
 ('South Korea', 158, 169, 384),
 ('Marilyn Monroe Rosenbergs', 171, 196, 383),
 ('Sugar Ray', 206, 215, 380),
 ('Panmunjom Brando', 217, 233, 380),
 ('The King and I" and "The Catcher', 236, 268, 388),
 ('England', 302, 309, 384),
 ('Marciano', 328, 336, 380),
 ('Liberace', 338, 346, 383),
 ('Santayana', 348, 357, 380),
 ('Joseph Stalin', 516, 529, 380),
 ('Malenkov', 531, 539, 380),
 ('Nasser', 541, 547, 380),
 ('Prokofiev Rockefeller', 552, 573, 380),
 ('Campanella', 575, 585, 380),
 ('Communist', 587, 596, 381),
 ('Roy Cohn', 602, 610, 380),
 ('Juan Peron', 612, 622, 380),
 ('Toscanini', 624, 633, 384),
 ('Dacron Dien Bien Phu', 635, 655, 380),
 ('Rock Around the Clock', 664, 685, 388),
 ('James Dean'

In [90]:
displacy.serve(doc, style="ent")

/Users/alexandervonschwerdtner/opt/miniconda3/envs/ba820/lib/python3.8/site-packages/spacy/displacy/__init__.py:98: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



## Custom NER Model

In [ ]:
################################## Custom NER
##
## we will build train our own NER model
##
##


In [1]:
# first, download the repo
# we are git cloning it into our session

# ! git clone https://github.com/Btibert3/BA820-Fall-2021.git

Cloning into 'BA820-Fall-2021'...
remote: Enumerating objects: 636, done.
remote: Counting objects: 100% (636/636), done.
remote: Compressing objects: 100% (489/489), done.
remote: Total 636 (delta 297), reused 429 (delta 119), pack-reused 0
Receiving objects: 100% (636/636), 17.36 MiB | 1.89 MiB/s, done.
Resolving deltas: 100% (297/297), done.
Updating files: 100% (164/164), done.


In [1]:
import json

In [3]:
# read in the pre-annotated dataset

path = "/Users/alexandervonschwerdtner/Desktop/BA820 - Unsupervised Machine Learning & Text Analytics/BA820-Fall-2021/apps/labelstudio/ner1.json"
with open(path, "r") as f:
  a = json.load(f)
  
a

[{'id': 100,
  'annotations': [{'id': 100,
    'completed_by': 2,
    'result': [],
    'was_cancelled': True,
    'ground_truth': False,
    'created_at': '2021-11-21T22:51:23.333397Z',
    'updated_at': '2021-11-21T22:51:23.333439Z',
    'lead_time': 1.23,
    'prediction': {},
    'result_count': 0,
    'task': 100,
    'parent_prediction': None,
    'parent_annotation': None}],
  'file_upload': '4e77d863-messages.csv',
  'drafts': [],
  'predictions': [],
  'data': {'text': "I don't have to be home to accept a delivery if I just order the Azalea Bonsai, do it? Is there any way I can specify that the delivery by left at my back door instead of on my porch? I prefer this when I'm at work."},
  'meta': {},
  'created_at': '2021-11-21T22:31:12.366323Z',
  'updated_at': '2021-11-21T22:31:12.366335Z',
  'project': 2},
 {'id': 99,
  'annotations': [{'id': 99,
    'completed_by': 2,
    'result': [],
    'was_cancelled': True,
    'ground_truth': False,
    'created_at': '2021-11-21T22:51:

> NOTE:  This was annotated using Label Studio!  The task was to identify product ids when supplied by customers in service requests.  This can be used to lookup information from a database for the customer service rep and expedite the process.

In [6]:
a = pd.read_csv('/Users/alexandervonschwerdtner/Desktop/BA820 - Unsupervised Machine Learning & Text Analytics/BA820-Fall-2021/data-challenges/12-news-classification/train.csv')


In [7]:
# needs to be in the form of 
# [ (doc, {“entities”: [ (start, end, label) ]}) ]
# this assumes one label per entity, but will put any
##
# label studio annotated NER training file in the shape that spacy needs!

TRAIN_DATA = []
for entry in a:
  doc = entry.get('data').get('text')
  result = entry.get("annotations")[0].get('result')
  if len(result) == 0:
    continue
  entities = []
  for r in result:
    ent = r.get('value')
    entities.append([ent.get('start'), ent.get('end'), ent.get('labels')[0]])
  TRAIN_DATA.append([doc, {'entities': entities}])


AttributeError: 'str' object has no attribute 'get'

In [12]:
# NOTE: this script is commonly found as the reference build the spacy file format

import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
      doc = nlp.make_doc(text)
      ents = []
      for start, end, label in annot["entities"]:
        # print(start, end, label)
        span = doc.char_span(start, end, label=label)
        if span is None:
            msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
            #warnings.warn(msg)
        else:
            ents.append(span)
      doc.ents = ents
      db.add(doc)
    db.to_disk(output_path)

# # this wrties the file to our session standard working directory
convert("en", TRAIN_DATA, "train.spacy")

In [13]:
# spacy requires/STRONGLY prefers (3+) that we use the explicit config system
# admittedly this adds a huge layer of complexity to the abstraction framework
# I suspect that this will become both easier and more transparent shortly

# the cli way to setup the model config
!python -m spacy init config --lang en --pipeline ner config.cfg --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [14]:
# copy the train.spacy as dev.spacy  -- not great, 
# but I didn't create enough labels for a dev set

! cp train.spacy dev.spacy

In [15]:
# tell spacy to train the model on our datasets and export the model

! python -m spacy train config.cfg                     \
                        --output ./                    \
                        --paths.train ./train.spacy    \
                        --paths.dev ./dev.spacy        \
                        --training.max_epochs 5        \
                        --training.eval_frequency 10

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-11-29 11:35:45,038] [INFO] Set up nlp object from config
[2021-11-29 11:35:45,058] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-11-29 11:35:45,065] [INFO] Created vocabulary
[2021-11-29 11:35:45,071] [INFO] Finished initializing nlp object
[2021-11-29 11:35:45,645] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     20.00    0.00    0.00    0.00    0.00
  0      10          0.86    514.44    0.00    0.00    0.00    0.00
  0      20          1.10     39.08    0.00    0.00    0.00    0.00
  0      30          0.73     28.78    0.00    0

In [17]:
# load in the custom model 
# remember, we only taught this model one thing 
# an NER model specific for one entity, a product id 

custom_nlp = spacy.load("model-best")
custom_nlp

In [20]:
# lets see this relative to our original NLP model

msg = "Brock teaches at Boston University"
nlp(msg).ents

(Brock, Boston University)

In [21]:
# and now with the custom model
custom_nlp(msg).ents
# why? because 

()

In [22]:
# this makes sense
# now, lets send a domain-specific message
# try 123423-23, ABQ12234
# play around with different reps, can see the product

msg2 = "Is the product (1232432) available for overnight shipping?"
[(ent.label_, ent.text) for ent in custom_nlp(msg2).ents]

# try 123423-23, ABQ12234
# play around with different reps, can see the product matters 
# with (), and doesnt like - in the product id

[('PRODUCT', '1232432')]

In [24]:
msg2 = "Is the product (123423-23) available for overnight shipping?"
[(ent.label_, ent.text) for ent in custom_nlp(msg2).ents]

[('PRODUCT', '123423-')]

In [25]:
msg2 = "Is the product (ABQ12234) available for overnight shipping?"
[(ent.label_, ent.text) for ent in custom_nlp(msg2).ents]

[('PRODUCT', 'ABQ12234')]

In [23]:
# but what about when we pass above into the PRETRAINED spacy model?
[(ent.label_, ent.text) for ent in nlp(msg2).ents]

[('CARDINAL', '1232432'), ('TIME', 'overnight')]

> If you attempt different values for the product above, you will see the generic spacy NER model flags the entity, but it can vary.  

__So what can we do?__

We _could_ add the new NER model into the pipeline.  WOrth noting, though, is that the order of the pipeline matters.  Naturally you would want to put this last, BUT, we saw that products are being flagged by the generic model.  If that is first in the pipeline, it will override our more accurately trained, Product Entitty.  This has all sorts of issues, but worth noting.

> This is a common problem when trying to think about updating models.  Models can forget.  One approach, and the one that the folks at explosion suggest is to train a net-new model, but one that blends a silver-standard dataset with the gold-standard dataset for the new entity.

- Create a training set from the existing model by passing in text and using the NER annotations as truth.  These are inferences, so considered "silver".
- Append to the dataset above the training data for the new entity
- train a completely new model, and use that.


> In a way, my endless rant on domain-specific applies here.




In [ ]:
###################################### YOUR TURN
##
## Using the dataset on Big Query `questrom.datasets.uber`
## extract the entities using spacy's learned NER model
## identify the top 5 locations (GPE)
## 
##

